In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
True


In [ ]:
# Clone mmcls repository and checkout to the 1.x branch
!git clone -b 1.x https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/

# Install MMClassification from source by mim
!pip install openmim
!mim install -e . 

In [ ]:
# Check MMClassification installation
import mmcls
print(mmcls.__version__)

1.0.0rc5


In [ ]:
# Download the dataset (cats & dogs dataset)
!wget https://www.dropbox.com/s/wml49yrtdo53mie/cats_dogs_dataset_reorg.zip?dl=0 -O cats_dogs_dataset.zip
!mkdir -p data
!unzip -q cats_dogs_dataset.zip -d ./data/

**After downloading and extraction,** we get "Cats and Dogs Dataset" and the file structure is as below:
```
data/cats_dogs_dataset
├── classes.txt
├── test.txt
├── val.txt
├── training_set
│   ├── training_set
│   │   ├── cats
│   │   │   ├── cat.1.jpg
│   │   │   ├── cat.2.jpg
│   │   │   ├── ...
│   │   ├── dogs
│   │   │   ├── dog.2.jpg
│   │   │   ├── dog.3.jpg
│   │   │   ├── ...
├── val_set
│   ├── val_set
│   │   ├── cats
│   │   │   ├── cat.3.jpg
│   │   │   ├── cat.5.jpg
│   │   │   ├── ...
│   │   ├── dogs
│   │   │   ├── dog.1.jpg
│   │   │   ├── dog.6.jpg
│   │   │   ├── ...
├── test_set
│   ├── test_set
│   │   ├── cats
│   │   │   ├── cat.4001.jpg
│   │   │   ├── cat.4002.jpg
│   │   │   ├── ...
│   │   ├── dogs
│   │   │   ├── dog.4001.jpg
│   │   │   ├── dog.4002.jpg
│   │   │   ├── ...
```

You can use shell command `tree data/cats_dogs_dataset` to check the structure.

### Support new dataset

We have two methods to support a new dataset in MMClassification.

The simplest method is to re-organize the new dataset as the format of a dataset supported officially (like `CustomDataset`). And you can also create a new dataset class. More details are in [the docs](https://mmclassification.readthedocs.io/en/dev-1.x/user_guides/dataset_prepare.html).

In this tutorial, for convenience, we have re-organized the cats & dogs dataset as the format of `CustomDataset`.

Besides image files, it also includes the training/validation/test annotation files. And every line includes an file path and the corresponding label.

```
...
cats/cat.3769.jpg 0
cats/cat.882.jpg 0
dogs/dog.3881.jpg 1
dogs/dog.3377.jpg 1
...
```

In [ ]:
%%writefile configs/mobilenet_v2/mobilenet_v2_1x_cats_dogs.py
_base_ = [
    '../_base_/models/convnext/convnext-base.py',
    '../_base_/schedules/imagenet_bs1024_adamw_swin.py',
    '../_base_/default_runtime.py',
]
# ---- model settings ----
# Here we use init_cfg to load pre-trained model.
# In this way, only the weights of backbone will be loaded.
# And modify the num_classes to match our dataset.

model = dict(
    backbone=dict(
        init_cfg = dict(
            _delete_=True,
            type='Pretrained', 
            checkpoint='https://download.openmmlab.com/mmclassification/v0/mlp-mixer/mixer-base-p16_3rdparty_64xb64_in1k_20211124-1377e3e0.pth', 
            prefix='backbone')
    ),
    head=dict(num_classes=2))

# ---- data settings ----
# We re-organized the dataset as `CustomDataset` format.
dataset_type = 'CustomDataset'
data_preprocessor = dict(
    mean=[124.508, 116.050, 106.438],
    std=[58.577, 57.310, 57.437],
    to_rgb=True,
)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', scale=224, backend='pillow'),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='PackClsInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='ResizeEdge', scale=256, edge='short', backend='pillow'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackClsInputs'),
]

train_dataloader = dict(
    batch_size=64,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='data/cats_dogs_dataset/training_set/training_set',
        classes=['cat', 'dog'],
        pipeline=train_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
)

val_dataloader = dict(
    batch_size=64,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='data/cats_dogs_dataset/val_set/val_set',
        classes=['cat', 'dog'],
        pipeline=test_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

test_dataloader = dict(
    batch_size=64,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_prefix='data/cats_dogs_dataset/test_set/test_set',
        classes=['cat', 'dog'],
        pipeline=test_pipeline,
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

# Specify the evaluation metric for validation and testing.
val_evaluator = dict(type='Accuracy', topk=1)
test_evaluator = val_evaluator




# Set training epochs and validate interval.
train_cfg = dict(by_epoch=True, max_epochs=10, val_interval=1)
# Use default settings for validation and testing
val_cfg = dict()
test_cfg = dict()

# ---- runtime settings ----
# Output training log every 10 iterations.
default_hooks = dict(logger=dict(interval=10))

# If you want to ensure reproducibility, set a random seed. And enable the
# deterministic option in cuDNN to further ensure reproducibility, but it may
# reduce the training speed.
randomness = dict(seed=0, deterministic=False)

Overwriting configs/mobilenet_v2/mobilenet_v2_1x_cats_dogs.py


In [ ]:
!python tools/train.py \
  configs/mobilenet_v2/mobilenet_v2_1x_cats_dogs.py \
  --work-dir work_dirs/mobilenet_v2_1x_cats_dogs

02/10 05:33:21 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.10 (default, Nov 14 2022, 12:59:47) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.6, V11.6.124
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.13.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_5

In [ ]:

!python tools/test.py configs/mobilenet_v2/mobilenet_v2_1x_cats_dogs.py work_dirs/mobilenet_v2_1x_cats_dogs/epoch_2.pth --dump result.pkl

02/10 05:31:21 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.10 (default, Nov 14 2022, 12:59:47) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.6, V11.6.124
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.13.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_5

In [ ]:
import mmengine

results = mmengine.load("result.pkl")
# Output the first samples' ground truth and prediction.
print('Ground truth:', results[3]['gt_label'])
print('Prediction:', results[3]['pred_label'])

Ground truth: {'label': tensor([0])}
Prediction: {'score': tensor([0.1041, 0.8959]), 'label': tensor([1])}


### Inference with a model

Sometimes we want to save the inference results on a image, just use the command below.

```shell
python demo/image_demo.py ${IMAGE_FILE} ${CONFIG_FILE} ${CHECKPOINT_FILE}
```


In [ ]:
!python demo/image_demo.py data/cats_dogs_dataset/training_set/training_set/cats/cat.1.jpg configs/mobilenet_v2/mobilenet_v2_1x_cats_dogs.py work_dirs/mobilenet_v2_1x_cats_dogs/epoch_2.pth

local loads checkpoint from path: work_dirs/mobilenet_v2_1x_cats_dogs/epoch_2.pth
{
  "pred_label": 0,
  "pred_score": 1.0,
  "pred_scores": [
    1.0,
    1.7860038781236653e-08
  ],
  "pred_class": "cat"
}
